In [ ]:
# 環境セットアップ
!pip install -q transformers peft accelerate bitsandbytes datasets trl
!pip install -q sentencepiece protobuf

In [ ]:
# インポート
import torch
import json
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)
from trl import SFTTrainer

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

In [ ]:
# Google Driveマウント（データアップロード用）
from google.colab import drive
drive.mount('/content/drive')

# データパス設定
DATA_PATH = Path("/content/drive/MyDrive/noteAI/data")
OUTPUT_PATH = Path("/content/drive/MyDrive/noteAI/models")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

## 設定

In [ ]:
# === トレーニング設定 ===
CONFIG = {
    # モデル
    "base_model": "rinna/japanese-gpt-neox-3.6b",

    # LoRA設定（2026年ベストプラクティス）
    "lora_r": 64,           # rsLoRAでは高rank推奨（8→64）
    "lora_alpha": 128,      # 通常 2*r
    "lora_dropout": 0.05,
    "use_rslora": True,     # ★ rsLoRA有効化
    "use_dora": True,       # ★ DoRA有効化（2024 ICML）

    # QLoRA設定
    "use_4bit": True,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",

    # トレーニング
    "max_seq_length": 256,
    "batch_size": 4,
    "gradient_accumulation": 4,
    "learning_rate": 2e-4,
    "num_epochs": 3,
    "warmup_ratio": 0.03,
    "weight_decay": 0.001,

    # 出力
    "output_name": "noteai-title-generator-v3",
}

print("設定:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## データ読み込み

In [ ]:
def load_training_data():
    """トレーニングデータを読み込み"""
    data = []

    # メインデータ
    main_file = DATA_PATH / "augmented" / "augmented_training.jsonl"
    if main_file.exists():
        with open(main_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    data.append(entry)
                except:
                    pass
        print(f"メインデータ: {len(data)}件")

    # Evol-Instructデータ
    evol_file = DATA_PATH / "processed" / "evol_instruct_data.jsonl"
    evol_count = 0
    if evol_file.exists():
        with open(evol_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    data.append(entry)
                    evol_count += 1
                except:
                    pass
        print(f"Evol-Instructデータ: {evol_count}件")

    print(f"合計: {len(data)}件")
    return data

raw_data = load_training_data()

In [ ]:
def format_training_example(entry):
    """トレーニング用フォーマットに変換"""

    # Evol-Instruct形式
    if "instruction" in entry:
        instruction = entry["instruction"]
        input_text = entry.get("input", "")
        output = entry.get("output", "")

        if input_text:
            prompt = f"### 指示:\n{instruction}\n\n### 入力:\n{input_text}\n\n### 出力:\n{output}"
        else:
            prompt = f"### 指示:\n{instruction}\n\n### 出力:\n{output}"

    # 通常形式（タイトルのみ）
    else:
        title = entry.get("title", "")
        category = entry.get("category", "")

        prompt = f"### 指示:\n以下のカテゴリに関する、読者を惹きつけるnote記事のタイトルを生成してください。\n\n### カテゴリ:\n{category}\n\n### タイトル:\n{title}"

    return {"text": prompt}

# データセット作成
formatted_data = [format_training_example(e) for e in raw_data]
dataset = Dataset.from_list(formatted_data)

print(f"\nデータセット: {len(dataset)}件")
print(f"\n例:")
print(dataset[0]["text"][:500])

## モデル読み込み（QLoRA）

In [ ]:
# 4-bit量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=CONFIG["use_4bit"],
    bnb_4bit_quant_type=CONFIG["bnb_4bit_quant_type"],
    bnb_4bit_compute_dtype=getattr(torch, CONFIG["bnb_4bit_compute_dtype"]),
    bnb_4bit_use_double_quant=True,
)

# モデル読み込み
print(f"モデル読み込み中: {CONFIG['base_model']}")
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# トークナイザー
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["base_model"],
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ モデル読み込み完了")

## rsLoRA + DoRA 設定

In [ ]:
# kbit training準備
model = prepare_model_for_kbit_training(model)

# LoRA設定（rsLoRA + DoRA）
lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,

    # ★ 2026年ベストプラクティス
    use_rslora=CONFIG["use_rslora"],     # rsLoRA: √r で除算
    use_dora=CONFIG["use_dora"],         # DoRA: 方向と大きさを分離

    # 全層にLoRAを適用（Sebastian Raschka推奨）
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ],
)

# PEFTモデル作成
model = get_peft_model(model, lora_config)

print("\n📊 LoRA設定:")
print(f"  rank: {CONFIG['lora_r']}")
print(f"  alpha: {CONFIG['lora_alpha']}")
print(f"  rsLoRA: {CONFIG['use_rslora']}")
print(f"  DoRA: {CONFIG['use_dora']}")

model.print_trainable_parameters()

## トレーニング

In [ ]:
# トレーニング引数
training_args = TrainingArguments(
    output_dir=str(OUTPUT_PATH / CONFIG["output_name"]),
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation"],
    learning_rate=CONFIG["learning_rate"],
    warmup_ratio=CONFIG["warmup_ratio"],
    weight_decay=CONFIG["weight_decay"],

    # 最適化
    optim="paged_adamw_32bit",
    fp16=True,
    gradient_checkpointing=True,

    # ログ
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,

    # その他
    report_to="none",
    push_to_hub=False,
)

# SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=CONFIG["max_seq_length"],
    dataset_text_field="text",
    packing=False,
)

print("✅ トレーナー準備完了")

In [ ]:
# トレーニング実行
print("🚀 トレーニング開始...")
trainer.train()
print("✅ トレーニング完了!")

In [ ]:
# モデル保存
save_path = OUTPUT_PATH / CONFIG["output_name"] / "final"
trainer.save_model(str(save_path))
tokenizer.save_pretrained(str(save_path))

print(f"✅ モデル保存: {save_path}")

## 推論テスト

In [ ]:
def generate_title(category: str, model, tokenizer, max_new_tokens: int = 64):
    """タイトルを生成"""
    prompt = f"### 指示:\n以下のカテゴリに関する、読者を惹きつけるnote記事のタイトルを生成してください。\n\n### カテゴリ:\n{category}\n\n### タイトル:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    title = generated.split("### タイトル:\n")[-1].strip()

    # 最初の行のみ
    title = title.split("\n")[0].strip()

    return title

# テスト
test_categories = ["副業", "英語学習", "プログラミング", "投資", "ブログ"]

print("\n🎯 生成テスト:")
print("=" * 50)

for cat in test_categories:
    title = generate_title(cat, model, tokenizer)
    print(f"[{cat}] {title}")

## 完了

モデルは Google Drive に保存されました:
- `/content/drive/MyDrive/noteAI/models/noteai-title-generator-v3/final/`

このモデルを使って推論するには `inference.py` を使用してください。